# Check in on Swarm

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import json
import sqlite3
from datetime import datetime
import requests
import json
import os

In [4]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [5]:
today = datetime.today().strftime("%Y%m%d")

In [11]:
foursquare_api_key = os.environ.get("foursquare_api_key")

---

### Find a venue

In [7]:
# Docs: https://developer.foursquare.com/reference/place-search

In [7]:
near = "Hollywood"
query = "CNN"

In [12]:
import requests

url = f"https://api.foursquare.com/v3/places/search?query={query}&near={near}"

headers = {
    "accept": "application/json",
    "Authorization": foursquare_api_key,
}

response = requests.get(url, headers=headers)

print(response.json())

{'results': [{'fsq_id': '4a66295bf964a52024c81fe3', 'categories': [{'id': 13031, 'name': 'Burger Joint', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burger_', 'suffix': '.png'}}, {'id': 13044, 'name': 'Frozen Yogurt Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/frozenyogurt_', 'suffix': '.png'}}, {'id': 13046, 'name': 'Ice Cream Parlor', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_', 'suffix': '.png'}}], 'chains': [], 'distance': 2914, 'geocodes': {'main': {'latitude': 33.995015, 'longitude': -118.39472}, 'roof': {'latitude': 33.995015, 'longitude': -118.39472}}, 'link': '/v3/places/4a66295bf964a52024c81fe3', 'location': {'address': '5622 Sawtelle Blvd', 'census_block': '060377026004003', 'country': 'US', 'cross_street': 'at Hannum Ave.', 'dma': 'Los Angeles', 'formatted_address': '5622 Sawtelle Blvd (at Hannum Ave.), Culver City, CA 90230', 'locality': 'Culver City', 'neighborhood': ['Beverlywood West'], 'postco

In [10]:
df = pd.DataFrame(response.json()["results"])

In [11]:
df[
    ["main.latitude", "main.longitude", "roof.latitude", "roof.longitude"]
] = pd.json_normalize(df["geocodes"])

In [12]:
df[
    [
        "address",
        "census_block",
        "country",
        "cross_street",
        "dma",
        "formatted_address",
        "locality",
        "neighborhood",
        "postcode",
        "region",
        "address_extended",
    ]
] = pd.json_normalize(df["location"])

In [13]:
df["categories"] = pd.json_normalize(df["categories"])

In [14]:
df[["id", "name", "icon.prefix", "icon.suffix"]] = pd.json_normalize(df["categories"])

In [15]:
df = df.drop(["categories", "geocodes", "location"], axis=1)

In [16]:
df.head()

,fsq_id,chains,distance,link,name,related_places,timezone,main.latitude,main.longitude,roof.latitude,roof.longitude,address,census_block,country,cross_street,dma,formatted_address,locality,neighborhood,postcode,region,address_extended,id,icon.prefix,icon.suffix
0,5b4bd8aacad1b60036b3c14f,[],3385,/v3/places/5b4bd8aacad1b60036b3c14f,Media Agency,{},America/Los_Angeles,33.930301,-118.382142,33.930301,-118.382142,2300 E Imperial Hwy,060379800131001,US,Douglas Street,Los Angeles,"2300 E Imperial Hwy (Douglas Street), El Segundo, CA 90245",El Segundo,[Westchester],90245,CA,NaN,11121,https://ss3.4sqi.net/img/categories_v2/building/default_,.png
1,5e7336d08bef4b8d18821c04,[],293,/v3/places/5e7336d08bef4b8d18821c04,Gym and Studio,{},NaN,33.921750,-118.415732,33.921750,-118.415732,436 Main St,060376201021016,US,NaN,Los Angeles,"436 Main St, El Segundo, CA 90245",El Segundo,[Westchester],90245,CA,NaN,18021,https://ss3.4sqi.net/img/categories_v2/building/gym_,.png
2,4fc52978e4b04526215b079b,[],2465,/v3/places/4fc52978e4b04526215b079b,Publisher,{},America/Los_Angeles,33.920079,-118.389993,33.920079,-118.389993,300 Continental Blvd,060379800131022,US,,Los Angeles,"300 Continental Blvd, El Segundo, CA 90245",El Segundo,[Sepulveda Boulevard],90245,CA,Ste 650,11143,https://ss3.4sqi.net/img/categories_v2/building/default_,.png
